In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_pickle('Cleaned_ed.pkl')

# MODELING- Binary Logistic Regression

In [3]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from keras.layers import Dense

/Users/leticiabeeck/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Create a Binary Logit Model

In [5]:
df['Student Achievement Rating'].value_counts()

Meeting Target        432
Approaching Target    204
Exceeding Target      157
Not Meeting Target      2
Name: Student Achievement Rating, dtype: int64

In [6]:
binary = {'Meeting Target':1,'Exceeding Target':1,
           'Not Meeting Target':0,'Approaching Target':0}
df['Student Achievement Rating'] = df['Student Achievement Rating'].map(binary)

In [7]:
boolean = {'Not Meeting Target': 1, 'Approaching Target': 2,'Meeting Target':3,
           'Exceeding Target':4}
df['Trust Rating'] = df['Trust Rating'].map(boolean)
df['Rigorous Instruction Rating'] = df['Rigorous Instruction Rating'].map(boolean)
df['Collaborative Teachers Rating']=df['Collaborative Teachers Rating'].map(boolean)
df['Supportive Environment Rating']=df['Supportive Environment Rating'].map(boolean)
df['Effective School Leadership Rating']=df['Effective School Leadership Rating'].map(boolean)
df['Strong Family-Community Ties Rating']=df['Strong Family-Community Ties Rating'].map(boolean)

In [8]:
df['Student Achievement Rating'].value_counts()

1    589
0    206
Name: Student Achievement Rating, dtype: int64

## Train Test Split 

In [9]:
df['City'] = [value if value in df['City'].value_counts().index[:4] else 'Other' for value in df['City']]

In [10]:
cats= ['City']
df.drop(['School Name', 'Location Code', 'Address (Full)'], axis=1, inplace=True)

In [11]:
X = df.drop('Student Achievement Rating',axis=1)
y = df['Student Achievement Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


X_train = pd.get_dummies(X_train, columns=cats, drop_first=True)
X_test = pd.get_dummies(X_test, columns=cats, drop_first=True)

In [12]:
[col for col in X_train.columns if col not in X_test.columns]

[]

In [13]:
edu_cats = {
    "PK,0K,01,02,03,04,05"     : "has_elementary",
    "PK,0K,01,02,03,04,05,06,07,08" : "has_middle_school",
    "0K,01,02,03,04,05,06,07,08,09,10,11,12"   : "has_high_school",
    '0K,01,02,03,04,05': "has_elementary",
    '0K,01,02,03,04,05,06,07,08':"has_middle_school",
    '04,05,06,07,08':'has_middle_school',
    '03,04,05':'has_elementary',
    '0K,01,02,03,04,05,06,07,08,SE':'has_middle_school', 
    'PK,0K,01,02,03,04,05,06':'has_elementary',
    'PK,0K,01,02,03,04,05,SE':'has_elementary',
    'PK,0K,01,02,03':'has_elementary', 
    '04,05':'has_elementary',
    '0K,01,02,03,04':'has_elementary', 
    '0K,01,02,03,04,05,06':'has_elementary', 
    'PK,0K,01,02':'has_elementary',
    'PK,0K,01,02,03,04':'has_elementary', 
    '02,03,04,05':'has_elementary',
    'PK,0K,01,02,03,04,05,07,08':'has_middle_school',
    'PK,0K,01,02,03,04,05,06,07,08,SE':'has_middle_school', 
    '06,07,08':'has_middle_school',
    '0K':'pre_school',
    '03,04,05,06':'has_elementary',
    '0K,01,02,03,04,05,SE':'has_elementary', 
    '01,05,06,07,08,K':'has_middle_school',
    '01,02,03,04,05,06,07,08':'has_middle_school',
    '0K,01,02,03,04,05,06,07':'has_middle_school'
}

X_train['Grade'] = df['Grades'].map(edu_cats)
X_test['Grade']=df['Grades'].map(edu_cats)

In [14]:
X_train['Grade'].unique()

array(['has_middle_school', 'has_elementary', 'has_high_school',
       'pre_school'], dtype=object)

In [15]:
X_train=pd.get_dummies(X_train, columns=['Grade'], drop_first=True)
X_test=pd.get_dummies(X_test,columns=['Grade'],drop_first=True)

In [16]:
X_train.drop('Grades',axis=1,inplace=True)

In [17]:
X_test.drop('Grades',axis=1,inplace=True)

In [18]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

0.7610062893081762

In [19]:
X_train.shape

(636, 156)

In [20]:
X_test.shape

(159, 155)

In [21]:
[col for col in X_train.columns if col not in X_test.columns]

['Grade_pre_school']

In [22]:
# Drop this since it only contains one observation

In [23]:
X_train.drop(['Grade_pre_school'],axis=1,inplace=True)

In [24]:
df['Student Achievement Rating'].mean()

0.740880503144654

In [25]:
lr = LogisticRegression()
lr.fit(X_test, y_test)
lr.score(X_test, y_test)

0.8930817610062893

In [26]:
lr.coef_

array([[-2.33242270e-02,  1.44729214e-02, -2.02725213e-02,
         1.55168527e-04, -2.27894579e-03,  9.22262841e-03,
        -7.05053423e-05, -2.18161425e-02,  2.45596849e-02,
         8.65416767e-03, -1.87598940e-02, -3.71741143e-02,
        -2.34271423e-02,  2.29300040e-02,  2.99956272e-02,
         2.31944717e-02,  1.55114348e-02,  0.00000000e+00,
         1.77088428e-02, -2.39670854e-02, -9.70137343e-02,
         7.38526125e-02,  1.04480630e-04, -3.42174333e-03,
        -5.20168451e-02, -3.36725685e-03,  3.06461446e-02,
         1.81015062e-02, -1.14567126e-01,  1.08149441e-01,
        -6.22375712e-02, -1.09819016e-02,  2.97985519e-04,
        -1.12265542e-02,  7.17156188e-02, -3.88190868e-02,
         6.32505550e-02,  4.70386171e-03, -2.56767968e-02,
         2.04865200e-02,  1.78114798e-02,  3.61786115e-03,
         1.34798268e-02,  5.51329628e-03,  6.83508617e-02,
         5.13417726e-02,  1.06290904e-01, -8.50793534e-03,
        -5.33571283e-02,  1.12323786e-01,  8.79849380e-0

In [44]:
results=pd.DataFrame(list(zip(X_train.columns,lr.coef_[0])))

In [28]:
results.sort_values(by=1)[0:10]

,0,1
70,Grade 5 Math 4s - Asian or Pacific Islander,-0.137321
78,Grade 6 ELA 4s - Black or African American,-0.128376
28,Grade 3 Math 4s - Black or African American,-0.114567
74,Grade 5 Math 4s - Economically Disadvantaged,-0.102811
20,Grade 3 ELA 4s - Asian or Pacific Islander,-0.097014
76,Grade 6 ELA 4s - All Students,-0.095143
88,Grade 6 Math 4s - Black or African American,-0.092857
118,Grade 8 ELA 4s - Black or African American,-0.089580
65,Grade 5 Math - All Students Tested,-0.068779
95,Grade 7 ELA - All Students Tested,-0.068232


In [41]:
results.sort_values(by=1,ascending=False)

,0,1
64,Grade 5 ELA 4s - Economically Disadvantaged,0.203708
58,Grade 5 ELA 4s - Black or African American,0.199899
94,Grade 6 Math 4s - Economically Disadvantaged,0.133921
56,Grade 5 ELA 4s - All Students,0.125141
68,Grade 5 Math 4s - Black or African American,0.119301
49,Grade 4 Math 4s - Hispanic or Latino,0.112324
29,Grade 3 Math 4s - Hispanic or Latino,0.108149
46,Grade 4 Math 4s - All Students,0.106291
89,Grade 6 Math 4s - Hispanic or Latino,0.103905
86,Grade 6 Math 4s - All Students,0.100837


**The DataFrame above shows the features that have a strong correlation with Student Achievement**